In [1]:
import os,json;
from arcgis.mapping import MapServiceLayer;
from arcgis.geometry import Geometry;
import pandas as pd;
import arcpy;

base = r'C:\Users\PDziemiela\Documents\GitHub\EPA-OC-USTGeocoder';


In [3]:
wlk = {};
for r,d,f in os.walk(base + os.sep + 'output'):
    for file in f:
        if file.endswith(".xlsx") and not file.startswith('~'):
            wlk[file] = False;
            
    for file in f:
        if file.endswith(".json") and os.path.splitext(file)[0] + '.xlsx' in wlk:
            wlk[os.path.splitext(file)[0] + '.xlsx'] = True;
        
rez = {};
for k,v in wlk.items():
    if v is True:
        rez[k] = {};
        
print("Found " + str(len(rez)) + " output Excel files to QA.");
for k,v in rez.items():
    print("  " + k);
    

Found 1 output Excel files to QA.
  AL_UST_template_data_only_20230110.xlsx


In [4]:
# Read the information from json companion file and check for problems

for k in list(rez.keys()):
    with open(base + os.sep + 'output' + os.sep + os.path.splitext(k)[0] + '.json') as f:
        invals = json.load(f);
    
    rez[k] = invals;
    print("reading " + os.path.splitext(k)[0] + '.json');
    

reading AL_UST_template_data_only_20230110.json


In [5]:
states = {}
tms = MapServiceLayer(
    url = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/State_County/MapServer/0'
);

for k in list(rez.keys()):
    state = rez[k]['input']['state'];
    
    if state is not None and state not in states:
        print("  fetching geometry for " + state);
        f = tms.query(
             where = "STUSAB = '" + state + "' "
            ,outSR = 4326
        );
        
        states[state] = Geometry(f.features[0].geometry).as_arcpy;


  fetching geometry for AL


In [6]:
def cleanFloat(pin):
    if pin is None or pin == "" or str(pin) == "nan":
        return None;
    return float(pin);

def cleanString(pin):
    if pin is None or pin == "" or str(pin) == "nan":
        return None;
    return pin;

sr = arcpy.SpatialReference(4326);

for k in list(rez.keys()):
    state = rez[k]['input']['state'];
    
    if state is not None:
    
        print("Analyzing " + k + " against state " + state + "...")
        src = arcpy.env.scratchGDB + os.sep + 'state';
        if arcpy.Exists(src):
            arcpy.Delete_management(src);

        arcpy.management.CreateFeatureclass(
             out_path          = arcpy.env.scratchGDB
            ,out_name          = 'state'
            ,geometry_type     = 'POLYGON'
            ,spatial_reference = sr
        );

        print("  Loading " + state + " into fgdb...",end='');
        with arcpy.da.InsertCursor(
             in_table    = src
            ,field_names = ['SHAPE@']
        ) as cursor:
            cursor.insertRow([states[state]]);
        print(str(arcpy.GetCount_management(src)[0]));

        sht = 0;

        root = 'lusttemp';
        trg1 = arcpy.env.scratchGDB + os.sep + root + '1';
        trg2 = arcpy.env.scratchGDB + os.sep + root + '2';
        trg3 = arcpy.env.scratchGDB + os.sep + root + '3';
        trg4 = arcpy.env.scratchGDB + os.sep + root + '4';
        trg5 = arcpy.env.scratchGDB + os.sep + root + '5';

        if arcpy.Exists(trg1):
            arcpy.Delete_management(trg1);

        if arcpy.Exists(trg2):
            arcpy.Delete_management(trg2);

        if arcpy.Exists(trg3):
            arcpy.Delete_management(trg3);

        if arcpy.Exists(trg4):
            arcpy.Delete_management(trg4);

        if arcpy.Exists(trg5):
            arcpy.Delete_management(trg5);

        print("  Loading " + k + " into fgdb...",end='');
        arcpy.conversion.ExcelToTable(
             Input_Excel_File = base + os.sep + 'output' + os.sep + k
            ,Output_Table     = trg1
        );
        print(str(arcpy.GetCount_management(trg1)[0]));

        print("  Limiting to geocoded records...",end='');
        arcpy.conversion.TableToTable(
             in_rows      = trg1
            ,out_path     = arcpy.env.scratchGDB
            ,out_name     = root + '2'
            ,where_clause = "gc_coordinate_source = 'Geocode' "
        );
        print(str(arcpy.GetCount_management(trg2)[0]));

        print("  Converting to feature class...",end='');
        arcpy.management.MakeXYEventLayer(
             table             = trg2
            ,in_x_field        = 'gc_longitude'
            ,in_y_field        = 'gc_latitude'
            ,out_layer         = trg3
            ,spatial_reference = sr
        );
        print(str(arcpy.GetCount_management(trg3)[0]));

        arcpy.conversion.FeatureClassToFeatureClass(
             in_features   = trg3
            ,out_path      = arcpy.env.scratchGDB
            ,out_name      = root + '4'
        );

        print("  Calculating Near values...");
        arcpy.analysis.GenerateNearTable(
             in_features    = trg3
            ,near_features  = src
            ,out_table      = trg5
            ,search_radius  = None
            ,method         = "GEODESIC"
            ,location       = "NO_LOCATION"
            ,angle          = "NO_ANGLE"
        );

        print("  Joining near values to feature class...");
        arcpy.management.JoinField(
             in_data    = trg4
            ,in_field   = 'OBJECTID'
            ,join_table = trg5
            ,join_field = 'IN_FID'
            ,fields     = ['NEAR_DIST']
        );

        print("  Hashing results...");
        results = {};
        with arcpy.da.SearchCursor(
             in_table    = trg4
            ,field_names = ['globalid','NEAR_DIST']
        ) as cursor:

            for row in cursor:
                results[row[0]] = row[1];

        print("  Updating " + k + "...");
        df = pd.read_excel(base + os.sep + 'output' + os.sep + k, sheet_name = sht);

        for idx,row in df.iterrows():
            src = cleanString(row['gc_coordinate_source']);

            if src == 'Geocode':
                lng = cleanFloat(row['gc_longitude']);
                lat = cleanFloat(row['gc_latitude']);
                ost = cleanString(row['gc_outside_state']);

                if  ost is None \
                and lng is not None and lat is not None:
                    guid = cleanString(row['globalid']);
                    
                    if guid not in results:
                        print(str(guid))
                        raise Exception("foo")

                    chk_dist = results[guid] * 0.000621371;
                    df.loc[idx,'gc_outside_state'] = chk_dist;

                    if chk_dist > 5:
                         df.loc[idx,'gc_status'] = 'Fail';

        print("  Writing out results");
        df.to_excel(
             excel_writer = base + os.sep + 'output' + os.sep + k
        );
    

Analyzing AL_UST_template_data_only_20230110.xlsx against state AL...
  Loading AL into fgdb...1
  Loading AL_UST_template_data_only_20230110.xlsx into fgdb...51137
  Limiting to geocoded records...19870
  Converting to feature class...19870
  Calculating Near values...
  Joining near values to feature class...
  Hashing results...
  Updating AL_UST_template_data_only_20230110.xlsx...
  Writing out results
